In [1]:
%pip install bert-tensorflow

Note: you may need to restart the kernel to use updated packages.


You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [9]:
from tensorflow import keras
import os
import re

import urllib.request
import tarfile
import re
import pandas as pd
import os

imdb_data_url  = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
imdb_data_file = "aclImdb_v1.tar.gz"

def read_imdb_data():
    member_re = re.compile(r"aclImdb/(test|train)/(pos|neg)/\d+_(\d+)\.txt")
    data = []
    with tarfile.open(imdb_data_file, "r") as tf:
        for member in tf.getmembers():
            if member.isfile():
                m = member_re.match(member.name)
                if m:
                    data_set = m.group(1)
                    label = m.group(2)
                    polarity = 1 if label == "pos" else 0
                    with tf.extractfile(member) as fh:
                        content = fh.read().decode('utf-8')
                    data.append([data_set, polarity, content])
    return pd.DataFrame(data=data, columns=['data_set', 'polarity', 'sentence'])

def load_imdb_data():
    if not os.path.isfile(imdb_data_file):
        print(f"Downloading: {imdb_data_url}")
        urllib.request.urlretrieve(imdb_data_url, imdb_data_file)
    return read_imdb_data()

imdb_data = load_imdb_data()


In [100]:
train0 = imdb_data[imdb_data.data_set.str.lower() == 'train']
test0 = imdb_data[imdb_data.data_set.str.lower() == 'test']

train = train0.sample(50, random_state = 11989)
test = test0.sample(50, random_state = 11989*2)

In [151]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 15:01:02.251002 22712 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [153]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.

def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [1]:
label_list

NameError: name 'label_list' is not defined

In [114]:
def BERT_Classification(train, test, DATA_COLUMN, LABEL_COLUMN, OUTPUT_DIR):
    
    bert_model_hub="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" #need to write it as a function later
    
    train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

    test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    
   
    tokenizer = create_tokenizer_from_hub_module()

    MAX_SEQ_LENGTH = 128
    # Convert our train and test features to InputFeatures that BERT understands.
    train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
    test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

    #compute train steps and warmup steps from batch size
    BATCH_SIZE = 32
    LEARNING_RATE = 2e-5
    NUM_TRAIN_EPOCHS = 3.0
    # Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
    WARMUP_PROPORTION = 0.1

    SAVE_CHECKPOINTS_STEPS = 500
    SAVE_SUMMARY_STEPS = 100
    # Compute # train and warmup steps from batch size
    
    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    #output directory and number of checkpoint steps to to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    #created model function
    model_fn = model_fn_builder(
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
   
    #create an input function for training; drop_remainder=True for TPUs
    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)
    
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    #create an input function for testing; drop_remainder=True for TPUs
    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    
    return {
        "Model_Evaluation":result_dict, 
        "Model": estimator
    }



In [115]:
start_time = datetime.now()
Bert_result=BERT_Classification(train, test, DATA_COLUMN, LABEL_COLUMN, OUTPUT_DIR)
print("time ", datetime.now() - start_time)
#50 samples 1 min on my laptop

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 14:27:07.697736 22712 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 50


I0523 14:27:08.255986 22712 run_classifier.py:774] Writing example 0 of 50


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Saving dict for global step 4: auc = 0.49033818, eval_accuracy = 0.46, f1_score = 0.6301369, false_negatives = 3.0, false_positives = 24.0, global_step = 4, loss = 0.73108596, precision = 0.45454547, recall = 0.8695652, true_negatives = 3.0, true_positives = 20.0


I0523 14:28:18.925968 22712 estimator.py:1979] Saving dict for global step 4: auc = 0.49033818, eval_accuracy = 0.46, f1_score = 0.6301369, false_negatives = 3.0, false_positives = 24.0, global_step = 4, loss = 0.73108596, precision = 0.45454547, recall = 0.8695652, true_negatives = 3.0, true_positives = 20.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4: movie\model.ckpt-4


I0523 14:28:18.940487 22712 estimator.py:2039] Saving 'checkpoint_path' summary for global step 4: movie\model.ckpt-4


time  0:01:14.859447


In [179]:
print(Bert_result['Model_Evaluation'])
BERTEstimator=Bert_result['Model']

{'auc': 0.49033818, 'eval_accuracy': 0.46, 'f1_score': 0.6301369, 'false_negatives': 3.0, 'false_positives': 24.0, 'loss': 0.73108596, 'precision': 0.45454547, 'recall': 0.8695652, 'true_negatives': 3.0, 'true_positives': 20.0, 'global_step': 4}


In [193]:
def getPrediction(in_sentences, estimator):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [194]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

In [195]:
predictions = getPrediction(pred_sentences, BERTEstimator)

INFO:tensorflow:Writing example 0 of 4


I0523 16:28:29.535923 22712 run_classifier.py:774] Writing example 0 of 4


INFO:tensorflow:*** Example ***


I0523 16:28:29.540937 22712 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0523 16:28:35.286102 22712 saver.py:1270] Restoring parameters from movie\model.ckpt-4


INFO:tensorflow:Running local_init_op.


I0523 16:28:36.586149 22712 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0523 16:28:36.746047 22712 session_manager.py:493] Done running local_init_op.


In [196]:
predictions

[('That movie was absolutely awful',
  array([-0.8132107, -0.585966 ], dtype=float32),
  'Positive'),
 ('The acting was a bit lacking',
  array([-0.7604427, -0.6300963], dtype=float32),
  'Positive'),
 ('The film was creative and surprising',
  array([-1.1095052 , -0.40006277], dtype=float32),
  'Positive'),
 ('Absolutely fantastic!',
  array([-0.8103343 , -0.58826363], dtype=float32),
  'Positive')]